### Correlation filter

#### This is the third step of the enrichment calculation

Some Org-IP samples may have very similar profiles, which may lead to underestimation of significance if outlier proteins overlap. 

To tackle this problem, enrichment values are pre-calculated using every other sample in the batch, and these enrichment values are correlated across samples.

The correlation values are then used to filter out highly correlated org-IP samples from being used in the significance test together.

In [1]:
import os, sys
from pathlib import Path
import pandas as pd

script_path = Path.cwd().parent.parent / "script"
data_path = Path.cwd().parent.parent / "data"
sys.path.append(str(script_path))
from pyseus import primary_analysis as pa
from pyseus import spatial_tools as st

### Load the imputed IP table
The correct datestamp is required to find the input files

In [2]:
%store -r timestamp USE_FROZEN
if USE_FROZEN:
    raise Exception("USE_FROZEN is true, you probably want to skip enrichment and proceed from Fig1")
print(f"Timestamp: {timestamp}")

outprefix = f"{timestamp}_QC_filter_impute"

outdir = Path.cwd() / "output"
preprocessing_out = outdir / f"preprocessing"

Timestamp: 2023-12-04


In [3]:
#### Load bait imputed tables (IPs)
IP_path = preprocessing_out / f"{outprefix}_imputed_table.csv"

try:
    bait_imputed_table = pd.read_csv(IP_path, header=[0, 1], index_col = 0)
except FileNotFoundError:
    print(f"File {IP_path} not found.\nPlease run 1.QC_filter_and_impute.ipynb first or specify the correct timestamp, current value is {timestamp}")
except pd.errors.ParserError:
    print(f"There was an error parsing the CSV file at {IP_path}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [4]:
# #check the WTs
# for i in bait_imputed_table.columns:
#     if "WT" in i[0]:
#         print(i)

### Correlation filter

setup the batches accordin to `2.Batch_selection.ipynb`

In [5]:
batches = [
    ["12", "13"],
    ["01", "03", "04", "05"],
    ["02", "06", "07"],
    ["14", "15", "17"],
    ["09", "10", "11"],
]

In [6]:
# imputed, preprocessed table from the earlier steps
grouped = bait_imputed_table.copy()

# initiate lists to add dataframes for concatenation at the end 
volcano_tables = []
enrichment_tables = []
volcano_tables_againstWT = []
enrichment_tables_againstWT = []

# create a folder to save the correlation tables
correlation_table_dir = outdir / "correlation_tables"
os.makedirs(correlation_table_dir, exist_ok=True)

# iterate through batches
for i, batch in enumerate(batches):
    print('batch: ' + str(i))
    new_cols = []
    # select all the samples from each batch
    for col in list(grouped):
        if col[0] == 'metadata':
            new_cols.append(col)
        # identifying each batch
        elif col[1].split('-')[0] in batch: # splitting the dash and seeing if that experiment is in the batch you're running
            new_cols.append(col) 
    selected = grouped[new_cols].copy()

    #print(list(selected))
    # initial calculation of complement enrichments to identify highly correlated samples
    analysis = pa.AnalysisTables(grouped_table=selected, auto_group=False)
    analysis.generate_export_bait_matrix()

    #analysis.select_wildtype_controls(wt_re='-WT') # select only WT as controls.
    analysis.select_matching_WT_control(wt_re='-WT') # select matching WT as controls. , this is confirmed to be the better than using only WT samples as the intial controls

    #print(analysis.exclusion_matrix) # uncomment to check the exclusion matrix
    analysis.exclusion_matrix.to_csv(correlation_table_dir / f"{timestamp}_negative_control_batch_{i}.csv", index=False)

    
    analysis.simple_pval_enrichment(std_enrich=False)
    analysis.convert_to_enrichment_table()
    complements = analysis.enrichment_table.copy()
    control_mat = analysis.exclusion_matrix.copy()

    enrichment_tables_againstWT.append(analysis.enrichment_table.copy()) # export the enrichments against WT
    volcano_tables_againstWT.append(analysis.simple_pval_table.copy()) # export the pvals against WT
    
    # Using the previous calculations, find highly correlated sample (filter is at >0.4)
    spatial = st.SpatialTables(
        preprocessed_table=selected,
        enrichment_table=complements,
        control_mat=control_mat)
    spatial.enrichment_corr_control_mat(corr=0.35)

    # save the correlation table for each batch
    spatial.corr_mat.to_csv(correlation_table_dir / f"{timestamp}_corr_table_batch_{i}.csv", index=False)  
    spatial.sample_corrs.to_csv(correlation_table_dir / f"{timestamp}_corr_val_table_batch_{i}.csv", index=True)
    
    # calculate the final pval and enrichment table for the batch
    spatial.new_corr_ARI(just_enrichment=True, labels=None, reference=None, std_enrich=False)

    # appending respective dataframes to the concatenation list
    volcano_tables.append(spatial.corr_pval_table)
    enrichment_tables.append(spatial.corr_enrichment_table)
    

batch: 0
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 1
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 2
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 3
P-val calculations..
Finished!
P-val calculations..
Finished!
batch: 4
P-val calculations..
Finished!
P-val calculations..
Finished!


The final step is to concatenate results from the batches. 

The enrichment table can be used for multiple purposes, including UMAP generation.

In [7]:
# concatenating the final enrichment table
enrichment_metadata = enrichment_tables[0].loc[:, ["metadata"]].copy()
enrichment_samples = [x.drop(["metadata"], axis=1, level=0) for x in enrichment_tables]

final_enrichments = pd.concat([enrichment_metadata] + enrichment_samples, axis=1)

# concatenating the final volcano table
volcano_metadata = volcano_tables[0].loc[:, ["metadata"]].copy()
volcano_samples = [x.drop(["metadata"], axis=1, level=0) for x in volcano_tables]

final_volcano = pd.concat([volcano_metadata] + volcano_samples, axis=1)

In [8]:
# saving the final tables to files
enrich_out_dir = outdir / "enrichment_and_volcano_tables"
os.makedirs(enrich_out_dir, exist_ok=True)

volcano_csv_path = enrich_out_dir / f"{timestamp}_volcano_table.csv"
enrichment_csv_path = enrich_out_dir / f"{timestamp}_enrichment_table.csv"

final_volcano.to_csv(volcano_csv_path)
final_enrichments.to_csv(enrichment_csv_path)

In [9]:
# check the enrichment table
final_enrichments

metadata  \
                                            Protein IDs   
0         A0A023T6R1;Q96A72;F5H6P7;F5H6N1;F5H3U9;F5H124   
1                          Q9Y5S9;A0A023T787;A0A0J9YW13   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6Z...   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...   
4                              Q99805;A0A024QYR8;B3KSG9   
...                                                 ...   
8594                                             X5D7P8   
8595                                             X5D8X9   
8596  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...   
8597                                             X5DQZ7   
8598                                             X6RHX1   

                                                                          \
                                   Majority protein IDs       Gene names   
0                       A0A023T6R1;Q96A72;F5H6P7;F5H6N1  FLJ10292;MAGOHB   
1                                     Q9Y5S9;A0A023T787       RBM8A;RBM8   
2     A0A0C4DFM1;A0A024QYR3;Q92544;B4DH88;B4DKC1;Q6ZTK5           TM9SF4   
3     A0A024QYR6;A0A1V0DNR7;A0A6G6A825;F6KD02;F6KD01...             PTEN   
4                              Q99805;A0A024QYR8;B3KSG9           TM9SF2   
...                                                 ...              ...   
8594                                             X5D7P8            STK39   
8595                                             X5D8X9          CNTNAP2   
8596  X5DQV1;X5DNI1;B3KV96;E9PD68;B3KXQ5;Q14194;B3KT...            CRMP1   
8597                                             X5DQZ7             GPX1   
8598                                             X6RHX1            SCYL3   

        sample                                                              \
     12-SEC61B     12-WT  13-RAB7A  12-G3BP1 12-TOMM20  12-LAMP1  13-RAB14   
0    -0.139405  0.497267  0.634090  0.208694 -0.654515 -0.266874  0.641974   
1    -0.631700  1.425600 -1.357550  0.621600 -2.248450 -0.588500 -0.984800   
2     4.080419 -3.924100  4.503702 -4.707100 -1.491024  3.421906  4.320300   
3    -0.874511  1.291439 -0.510095 -0.198848 -0.365514  0.959333 -0.412718   
4     7.104435 -6.935046  8.938121 -5.933239 -4.523732  5.796589  8.061546   
...        ...       ...       ...       ...       ...       ...       ...   
8594  0.967365 -0.679866  1.069567  0.137632  1.616812  0.362894  0.546989   
8595 -0.021208  0.374195  0.625338 -0.771679 -0.213794  0.194316  0.356751   
8596  0.491496 -0.357358 -0.820043  0.044803  0.351342  0.040040  1.146720   
8597  0.184980  0.766431  1.318104  0.794309  2.120314 -0.876673  0.025663   
8598  0.105110 -0.705130 -0.164388  0.293702 -0.494049  0.083745  0.466224   

      ...                                                              \
      ... 10-TOMM20  10-AP2B1  10-VPS35   11-EEA1 11-GPR107 11-CEP350   
0     ... -0.682980 -0.339033 -0.185583 -0.681700 -0.615860  0.160560   
1     ... -2.969400 -0.336600 -0.639400 -1.618800 -2.043600  0.895300   
2     ... -2.685600  0.620900  0.541700 -1.672800  5.885700  0.397300   
3     ...  1.041807  0.167305  0.562388 -1.261271 -0.394072  0.064974   
4     ... -2.382900  1.823800  2.232500  2.055000  5.827450 -0.438800   
...   ...       ...       ...       ...       ...       ...       ...   
8594  ... -0.078216 -0.165316 -1.577387  0.023261 -0.641255  0.428594   
8595  ... -0.874136 -0.403408 -0.825813  1.104272 -0.473069  1.947072   
8596  ... -0.216215 -0.039671 -0.462787  0.722372 -0.688941 -0.044831   
8597  ...  3.216147  0.973707 -0.923395 -1.142611  0.450772  0.828155   
8598  ... -0.036435  0.270969  0.693942 -0.660514 -1.233455  0.931407   

                                              
     09-ATG101   09-PEX3 11-SEC31A   10-RTN4  
0     3.270767 -0.086078 -1.159713 -0.158394  
1    -0.728450 -0.496950  0.979850 -1.899050  
2     2.393000 -0.092300  1.688300  1.852200  
3     0.978763  0.488340 -0.652074  0.656888  
4    -0.412350  0.016550  1.179800  1.52495